<img src="https://drive.google.com/uc?id=1-cL5eOpEsbuIEkvwW2KnpXC12-PAbamr" style="Width:1000px">

# Introduction to `RandomForest`


The goal of this exercise will be to explore  `RandomForest` for Classification. We will use the same dataset about earthquakes in California that you used yesterday. We will show that `RandomForest` does better at classifying this data than the `neural networks`! And what's more, it is faster to train and much easier.

# Importing the data
I have already prepared and split the data for you, so there is no need to do any data preparation for this exercise. The preparation follows exactly my solution to the exercise from yesterday. Simply run the code to load the data.

In [ ]:
from nbta.utils import download_data
download_data(id='1H_uTkFdNkY1FXllK5CmOjPpcqqd_PzkZ')

Now let's read the file into a `train` and `test` set:

In [ ]:
# Reading the files
import pandas as pd

X_train = pd.read_csv('raw_data/X_train_prep.csv')
X_test = pd.read_csv('raw_data/X_test_prep.csv')

y_train = pd.read_csv('raw_data/y_train.csv').values.ravel() # Ravel is used to change a column vector into a row vector
y_test = pd.read_csv('raw_data/y_test.csv').values.ravel()

In [ ]:
X_train

### Creating a smaller set

Training on 225k+ instances will take time. So we will experiment by creating a smaller version of our `X_train`. This is something to do with caution, as our algorithm will learn from the statistics of this train set and not from the entire dataset. But it will be much faster for some of the tests I want you to do. And we will eventually use the entire `X_train` and `X_test` for our algorithm.

Let's resample our `X_train` to take only 20% of the dataset. The easiest it to create a new dataframe (`data`) that contains both `X_train` and `y_train` and sample it randomly. This gives us `X_train_small` and `X_test_small` (still containing about 45k instances):

In [ ]:
data = X_train.copy()
data['y'] = y_train.copy()

data = data.sample(frac=.2,random_state = 42)

X_train_small = data.drop(columns='y')
y_train_small = data.y.copy()

# GridSearch different algorithms

Create a simple pipeline that contains a `classifier` parameter (you can set it up to be `RandomForestClassifier()`. Then, create a parameter grid that will automatically test this pipeline by switching the `classifier` between these algorithms: 
* `SVC()`
* `DecisionTreeClassifier()`
* `RandomForestClassifier()`
* `LogisticRegression(max_iter=3000)` (the standard `max_iter` will throw errors)
* `KNeiborsClassifier()`

Feel free to add any other classifier to the list if you want. Run the `GridSearchCV` with `cv=5` and `scoring="precision"`. Once the algorithm is fitted with `X_train_small` and `y_train_small`, check for the `best_score_`, and save the `best_params_` into a variable called `best_parameters`: so, which model won?

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

model_pipe = Pipeline([('classifier',RandomForestClassifier())])

param_grid = [
    {'classifier': [SVC(), DecisionTreeClassifier(), RandomForestClassifier(), LogisticRegression(max_iter=3000), KNeighborsClassifier()]}]

grid_search = GridSearchCV(model_pipe, param_grid, cv=5,
                           scoring='precision', n_jobs=-1)

In [ ]:
grid_search.fit(X_train_small, y_train_small)

In [ ]:
best_parameters = grid_search.best_params_
best_parameters

In [ ]:
best_score = grid_search.best_score_
best_score

In [ ]:
best_parameters

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('best_algorithm',
                         model = best_parameters,
                         best_score = best_score
)

result.write()
print(result.check())

# A critical look

You should have obtained a 95% precision with only 20% of the data we used to train our `MLPClassifier` last day. This is a very good performance! But remember what we learned about metrics on day 2? One metric alone can hide problems in our model.

Let's do the following here: calculate the `precision_score` of your model on the `X_test`, and then the `recall_score`. Save both of these into variables `precision_score_rf` and `recall_score_rf`.  What do you conclude?

In [ ]:
from sklearn.metrics import precision_score, recall_score

best_model = grid_search.best_estimator_
precision_score_rf = precision_score(y_test, best_model.predict(X_test))
recall_score_rf = recall_score(y_test, best_model.predict(X_test))

In [ ]:
precision_score_rf

In [ ]:
recall_score_rf

<details><summary>Conclusions</summary>
Whilst the precision score is reasonably high, the recall score is <85%. If we pushed the training more, there is a risk that we would overfit to the data even more and end up with a very low recall. To avoid that, from now on we will use `f1` as our metric in `GridSearchCV`. This ensures that we improve both `precision` and `recall` together.
</details>

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('balance_score',
                         precision_score = precision_score_rf,
                         recall_score = recall_score_rf
)

result.write()
print(result.check())

 Narrowing our Hyperparameter Space with RandomSearchCV

As you have discovered, for our dataset at least, `RandomForest` outperforms all other models.  Very often, `ensemble` methods will outperform other approaches, and on tabular data, it will even outperform deep-learning (and is much easier and quicker to train). So this needs to be part of your arsenal of methods!

What we will do now is try to fine-tune our `RandomForest` algorithm by using `RandomizedSearchCV`. We have also established that if we want to avoid overfitting to `precision` (which would mean a very low `recall`, i.e. only picking up a few positive classes and leaving a lot behind), we are better off using `f1` as our metric for `RandomizedSearchCV`. I suggest you try the following combination of hyperparameters into `GridSearchCV`:

* `max_depth` randomly between 1 and 50
* `n_estimator` randomly between 100 and 500
* `min_sample_split` randomly between 2 and 10
* `max_features` randomly between 1 and 8

This time, use the full `X_train` and `y_train` (not the smaller version). Run your `RandomizedSearchCV` with `cv=5`, `n_iter=10`(training is long because the dataset is large, and for this exercise we don't want to spend 30 minutes), scoring for accuracy, `random_state=42`,  and setting your `n_jobs=-1` to parallelize the work. Then, save the `best_score_` in a variable called `best_score` and the best algorithm (`best_estimator_`) as `rf_model`.

Then, use your `X_test` to calculate the `recall_score` and `precision_score` (save these into the `rf_recall` and `rf_precision`, respectivly. Did you beat the standard hyperparameters?

**Warning**: This code took a full 5 minutes to run on my machine, so be patient (or go take a break when it runs). Long running time for machine learning is normal, and you will learn this even more when you do deep-learning.

In [ ]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_distrib = [
    {'classifier__max_depth': randint(low=1, high=50),
     'classifier__n_estimators': randint(low=100, high=500),
    'classifier__min_samples_split': randint(low=2, high=10),
    'classifier__max_features': randint(low=1, high=8)}]

rnd_search = RandomizedSearchCV(model_pipe, param_distributions=param_distrib, cv=5, n_iter=10,
                           scoring='f1', n_jobs=-1, random_state=1)

In [ ]:
rnd_search.fit(X_train, y_train)

In [ ]:
rnd_search.best_params_

In [ ]:
best_score = rnd_search.best_score_
best_score

In [ ]:
rf_model = rnd_search.best_estimator_

In [ ]:
rf_precision = precision_score(y_test,rf_model.predict(X_test))
rf_precision

In [ ]:
from sklearn.metrics import recall_score

rf_recall = recall_score(y_test,rf_model.predict(X_test))
rf_recall 

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('randomized_search_score',
                         rf_precision = rf_precision,
                         rf_recall = rf_recall
)

result.write()
print(result.check())

# Why Ensemble Learning Works

The way your `RandomForestClassifier` algorithm reached a decision is by pooling the predictions of many individual `DecisionTreeClassifiers` together: the class that was voted most often was selected. Ensemble method work because they rely on the fact that the classification error of weak learners (such as `DecisionTreeClassifiers`) is random, and thus if the experiment is repeated often the error distribution will be guassian, with the mean centered on the correct value.

We can convince ourselves of that by looking at individual `DecisionTreeClassifiers` and pooling their results.

First, create a new pipeline (I call it `dtree`) that combines your `preproc` pipeline with a `DecisionTreeClassifier()` algorithm. Then, run a `cross_validate` loop on `dtree` using the `X_train_small` and `y_train_small` (for speed), and a `cv=200`. Save the result of this into a variable (`cv`) and plot the histogram of the `test_score`. What do you observe?

In [ ]:
from sklearn.model_selection import cross_validate

cv = cross_validate(DecisionTreeClassifier(), X_train_small, y_train_small, cv=100)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(cv['test_score']);

<details><summary>🌳 Observations</summary><br>
    We can see that the distribution of the scores is more or less Gaussian, but each individual tree performs very differently, some with very high scores (overfitting) and some with very low scores. Most of the trees have a precision close to 98%</details>

## Ensembling the trees

We can do a pseudo-ensemble very simply by using the `cross_val_predict` function located in the same module as `cross_validate`. The `cross_val_predict` function will return the average prediction (in our case, class) during cross validation. Run `cross_val_predict` using the same parameters as for `cross_validate` above (use `X_train_small` and `y_train_small`), and save the results in a variable (say, `cv_predict`). Then, run an `accuracy_score` function using `y_train` and `cv_predict`, and save this as `cv_score`: are we close to the `RandomForestClassifier` score?

In [ ]:
from sklearn.model_selection import cross_val_predict

cv_predict = cross_val_predict(DecisionTreeClassifier(), X_train_small, y_train_small, cv=200)

In [ ]:
from sklearn.metrics import precision_score

cv_score = precision_score(y_train_small, cv_predict)
cv_score

<details><summary>🌳 Observations</summary><br>
    The score should be around 84-85% accuracy, which is less than our <code>RandomForestClassifier</code>. Ultimately, <code>RandomForestClassifier</code> will work better because of the more advanced <code>bootstrapping</code> of the data and the ability to randomly select different <code>features</code> for each tree: this increases the variance of the trees, and thus results in stronger predictive performance than what we can so with a simple cross-validation</details>

## Feature Importance

With `DecisionTreeClassifiers`, it is very easy to see how a decison is made: simply follow the branches of the decision trees, and it will be obvious why a given sample belongs to a particular class. However, by pooling together many trees, `RandomForestClassifiers` loose this simple interpretability. The field of `Explainable AI` is an expanding one: in many fields (think law, medicine, finance) being able to explain why you reach a certain decision is just as important as the decision itself.

Explainable AI is beyond the the topic of this module, but we can at least explore the `feature importance` parameters of `RandomForest`, which is based on the `Gini impurity` index and a weighted factor based on the `probability of a given class being reached` on the tree. More details about the mathematics of this <a href="https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3#:~:text=Feature%20importance%20is%20calculated%20as,the%20more%20important%20the%20feature.">can be found in this excellent Medium post</a>. 

Using your previous `rf_model` (best estimator) that you saved above, access the `classifier` (remember: `GridSearchCV` and `RandomSearchCV` results can be accessed as dictionaries) output the `feature_importances_` values: a bigger value means more importance.

To know what value corresponds to what variable is easy: create a new `pd.Series` called `f_imp` wiht the `data` being the `feature_importance_` and the columns being the columns of `X_train`, and sort this series in descending order.

Now you know what features contribute to the decision, and what feature do not!


In [ ]:
f_imp = pd.Series(rf_model['classifier'].feature_importances_, index=X_train.columns).sort_values(ascending=False)
f_imp

## Note on decision trees probabilities

As with `LogisticRegression`, you can output a classification probability from `DecisionTrees` by using the `predict_proba()` function. Try this below for your `X_test` - you should see, for each sample, a list of 2 probabilities (one per class) and the largest is the one selected:

In [ ]:
rf_model.predict_proba(X_test)

However, <strong style="color:red">unlike LogisticRegression</strong> the probability of `DecisionTrees` is **not calibrated**: this means it represent the probability of a sample belonging to the given node of the tree, but **NOT** the probability of the sample distribution. `LogisticRegression` in that sense is better, though there has been some efforts to improve the probabilities of uncalibrated model by, for instance, combining logistic regression and decision trees (see <a href="https://gdmarmerola.github.io/probability-calibration/"> a very interesting post about this here</a>).

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('trees',
                         score = cv_score, 
                         importance = f_imp
)

result.write()
print(result.check())

# 🏁 You are all done!